In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Softmax
from tensorflow.keras import optimizers
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from numpy.random import seed
import tensorflow 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
housingData = pd.read_csv('/kaggle/input/california-housing-prices/housing.csv')

In [ ]:
housingData.head() 

In [ ]:
print(housingData.shape)

In [ ]:
housingData.describe()

In [ ]:
housingData.hist(bins=50,figsize=(50,45))
plt.show()

In [ ]:
corr_matrix = housingData.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
housingData.shape
housingData['ocean_proximity'] = housingData['ocean_proximity'].astype('category')
housingData['ocean_proximity'] = housingData['ocean_proximity'].cat.codes

continuous = ['population', 'median_income', 'median_house_value', 'latitude', 'longitude','housing_median_age','total_rooms','total_bedrooms','households','housing_median_age','ocean_proximity']
scaler = MinMaxScaler()

for var in continuous:
    housingData[var] = housingData[var].astype('float64')
    housingData[var] = scaler.fit_transform(housingData[var].values.reshape(-1, 1))


housingData.drop(['population', 'latitude', 'longitude','total_bedrooms','households'],axis=1,inplace=True)

X_data=housingData.drop('median_house_value',axis=1)
Y_data=housingData['median_house_value'].copy()

X=X_data.values
Y=Y_data.values

print(X)
print("************")
print(Y)

In [ ]:
from sklearn import preprocessing


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,)

In [ ]:

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)


In [ ]:
#finish all data things 

#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

from keras.models import Sequential
from keras.layers import Dense

In [ ]:

model = Sequential([
    Dense(10, activation='relu',input_shape=(4,)),
    Dense(8,  activation='relu'),
    Dropout(0.0),
    Dense(1,activation='relu'),
])

opt= keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer='adam', loss="mean_squared_error",metrics=['accuracy'])

model.summary()

In [ ]:
training = model.fit(X_train, Y_train, epochs=100, batch_size=40, validation_split=0.2, verbose=0)

In [ ]:
val_acc = np.mean(training.history['val_accuracy'])
print("val_accuracy ","%.3f"%(val_acc*100),"%")

In [ ]:
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
def create_model(lyrs=[10,8], act='relu', opt= keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999, amsgrad=False), dr=0.0):
    
    # set random seed for reproducibility
    seed(42)
    tensorflow.random.set_seed(42)
    
    model = Sequential()
    
    # create first hidden layer
    model.add(Dense(lyrs[0], input_dim=X_train.shape[1], activation=act))
    
    # create additional hidden layers
    for i in range(1,len(lyrs)):
        model.add(Dense(lyrs[i], activation=act))
    
    # add dropout, default is none
    model.add(Dropout(dr))
    
    # create output layer
    model.add(Dense(1, activation='relu'))  # output layer
    opt.lr=0.0002
    model.compile(loss="mean_squared_error", optimizer=opt, metrics=['accuracy'])
    
    return model

model = create_model()
model.summary()


In [ ]:

from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=40, verbose=0)


In [ ]:
drops = [0.0, 0.01, 0.05,0.1]
param_grid = dict(dr=drops)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1)
grid_result = grid.fit(X_train, Y_train)


In [ ]:

print("Best: %f using %s" % (grid_result.best_score_*100, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
activation = ['linear','sigmoid','relu']
param_grid = dict(act=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=0)
grid_result = grid.fit(X_train, Y_train)


In [ ]:

print("Best: %f using %s" % (grid_result.best_score_*100, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
def create_model(lyrs=[10,8], act='relu', opt= keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999, amsgrad=False), dr=0.0):
    
    # set random seed for reproducibility
    seed(42)
    tensorflow.random.set_seed(42)
    
    model = Sequential()
    
    # create first hidden layer
    model.add(Dense(lyrs[0], input_dim=X_train.shape[1], activation=act))
    
    # create additional hidden layers
    for i in range(1,len(lyrs)):
        model.add(Dense(lyrs[i], activation=act))
    
    # add dropout, default is none
    model.add(Dropout(dr))
    
    # create output layer
    model.add(Dense(1, activation='relu'))  # output layer
    #opt.lr=0.0002
    model.compile(loss="mean_squared_error", optimizer=opt, metrics=['accuracy'])
    
    return model

model = create_model()
model.summary()


In [ ]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=40, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Nadam']

param_grid = dict(opt=optimizer)

# search the grid

grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1)
grid_result = grid.fit(X_train, Y_train)



In [ ]:

print("Best: %f using %s" % (grid_result.best_score_*100, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:

model = create_model(lyrs=[10,8], act='linear', opt=keras.optimizers.Nadam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999) , dr= 0.1)


print(model.summary())

training = model.fit(X_train, Y_train, epochs=50, batch_size=32, 
                     validation_split=0.2, verbose=0)


In [ ]:
val_acc = np.mean(training.history['val_accuracy'])
print("val_accuracy ","%.3f"%(val_acc*100),"%")

plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:

scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))